In [1]:
import os

In [2]:
os.chdir('../')

In [3]:
pwd

'f:\\projects\\project-9 kidney disease\\kidney-disease-classification'

In [ ]:
from dataclasses import dataclass
from pathlib import Path

# --- Cell 2: Entity ---
@dataclass
class TrainingConfig:
    root_dir: Path
    trained_model_path: Path
    updated_base_model_path: Path
    training_data: Path
    params_epochs: int
    params_batch_size: int
    params_is_augmentation: bool
    params_image_size: list

In [ ]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories

In [ ]:
# --- Cell 3: Configuration Manager ---
class ConfigurationManager:
    def __init__(self, config_filepath=CONFIG_FILE_PATH, params_filepath=PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])

    def get_training_config(self, model_name: str) -> TrainingConfig:
        training = self.config.training
        prepare_base_model = self.config.prepare_base_model[model_name.lower()]
        params = self.params
        
        create_directories([Path(training.root_dir)])

        return TrainingConfig(
            root_dir=Path(training.root_dir),
            trained_model_path=Path(training[model_name.lower()].trained_model_path),
            updated_base_model_path=Path(prepare_base_model.updated_path),
            training_data=Path(self.config.data_ingestion.unzipped_data_dir),
            params_epochs=params[model_name]['EPOCHS'],
            params_batch_size=params.BATCH_SIZE,
            params_is_augmentation=params[model_name]['AUGMENTATION'],
            params_image_size=params.IMAGE_SIZE
        )

In [ ]:
import tensorflow as tf

In [ ]:
# --- Cell 4: Training Component ---
class Training:
    def __init__(self, config: TrainingConfig):
        self.config = config

    def get_base_model(self):
        self.model = tf.keras.models.load_model(self.config.updated_base_model_path)

    def train_valid_generator(self):
        datagenerator_kwargs = dict(
            rescale = 1./255,
            validation_split=0.20 # This handles the "separation" of data
        )

        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:2],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear"
        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(**datagenerator_kwargs)

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )

        if self.config.params_is_augmentation:
            train_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
                rotation_range=20,
                horizontal_flip=True,
                width_shift_range=0.1,
                height_shift_range=0.1,
                shear_range=0.1,
                zoom_range=0.1,
                **datagenerator_kwargs
            )
        else:
            train_datagenerator = valid_datagenerator

        self.train_generator = train_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="training",
            shuffle=True,
            **dataflow_kwargs
        )

    def train(self):
        self.model.fit(
            self.train_generator,
            epochs=self.config.params_epochs,
            validation_data=self.valid_generator
        )
        self.model.save(self.config.trained_model_path)

In [ ]:
# --- Cell 5: Averaging Ensemble Logic ---
import numpy as np

def run_ensemble_averaging(models_list, test_generator):
    """
    Takes a list of trained models and averages their predictions.
    """
    all_preds = []
    for model_path in models_list:
        model = tf.keras.models.load_model(model_path)
        preds = model.predict(test_generator)
        all_preds.append(preds)
    
    # Simple Averaging
    avg_preds = np.mean(all_preds, axis=0)
    final_classes = np.argmax(avg_preds, axis=1)
    return final_classes

In [ ]:
# --- Cell 6: Execution Pipeline ---
try:
    config = ConfigurationManager()
    model_names = ['DenseNet121', 'ResNet50', 'EfficientNetB0']
    trained_model_paths = []

    for name in model_names:
        print(f">>> Starting training for {name}")
        training_config = config.get_training_config(name)
        trainer = Training(config=training_config)
        trainer.get_base_model()
        trainer.train_valid_generator()
        trainer.train()
        trained_model_paths.append(training_config.trained_model_path)
    
    # After training all 3, run the averaging
    print(">>> Running Averaging Ensemble Evaluation...")
    # Use the last trainer's valid_generator as a proxy for test data
    ensemble_results = run_ensemble_averaging(trained_model_paths, trainer.valid_generator)
    print(f"Ensemble Predictions Complete: {ensemble_results[:10]}")

except Exception as e:
    raise e